In [1]:
import numpy as np

In [2]:
A = np.array([[4, 0, 0],
              [1, 2, 3],
              [0, 0, 5]])

In [3]:
r = np.array([2, 0, -1])


In [4]:
rows, column_indices = np.ogrid[:A.shape[0], :A.shape[1]]

In [5]:
rows

array([[0],
       [1],
       [2]])

In [6]:
column_indices

array([[0, 1, 2]])

In [9]:
r[r < 0] += A.shape[1]
A.shape[1]

3

In [8]:
r

array([2, 0, 2])

In [10]:
column_indices = column_indices - r[:, np.newaxis]

In [12]:
 r[:, np.newaxis]

array([[2],
       [0],
       [2]])

In [11]:
column_indices

array([[-2, -1,  0],
       [ 0,  1,  2],
       [-2, -1,  0]])

In [13]:
result = A[rows, column_indices]


In [14]:
result

array([[0, 0, 4],
       [1, 2, 3],
       [0, 5, 0]])